# **Identifying Key Entities in Recipe Data**


**Business Objective**:
The goal of this assignment is to train a Named Entity Recognition (NER) model using Conditional Random Fields (CRF) to extract key entities from recipe data. The model will classify words into predefined categories such as ingredients, quantities and units, enabling the creation of a structured database of recipes and ingredients that can be used to power advanced features in recipe management systems, dietary tracking apps, or e-commerce platforms.

### **Data Description**
The given data is in JSON format, representing a **structured recipe ingredient list** with **Named Entity Recognition (NER) labels**. Below is a breakdown of the data fields:

```json
[
    {
        "input": "6 Karela Bitter Gourd Pavakkai Salt 1 Onion 3 tablespoon Gram flour besan 2 teaspoons Turmeric powder Haldi Red Chilli Cumin seeds Jeera Coriander Powder Dhania Amchur Dry Mango Sunflower Oil",
        "pos": "quantity ingredient ingredient ingredient ingredient ingredient quantity ingredient quantity unit ingredient ingredient ingredient quantity unit ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient"
    },
    {
      "input": "2-1/2 cups rice cooked 3 tomatoes teaspoons BC Belle Bhat powder 1 teaspoon chickpea lentils 1/2 cumin seeds white urad dal mustard green chilli dry red 2 cashew or peanuts 1-1/2 tablespoon oil asafoetida",
      "pos": "quantity unit ingredient ingredient quantity ingredient unit ingredient ingredient ingredient ingredient quantity unit ingredient ingredient quantity ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient quantity ingredient ingredient ingredient quantity unit ingredient ingredient"
    }
]


| **Key**  | **Description**  |
|----------|-----------------|
| `input`  | Contains a raw ingredient list from a recipe. |
| `pos`    | Represents the corresponding part-of-speech (POS) tags or NER labels, identifying quantities, ingredients, and units. |


## **1** Import libraries

#### **1.1** Installation of sklearn-crfsuite

sklearn-crfsuite is a Python wrapper for CRFsuite, a fast and efficient implementation of Conditional Random Fields (CRFs). It is designed to integrate seamlessly with scikit-learn for structured prediction tasks such as Named Entity Recognition (NER), Part-of-Speech (POS) tagging, and chunking.

In [4]:
# installation of sklearn_crfsuite
!pip install sklearn_crfsuite==0.5.0

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\v2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


#### **1.2** Import necessary libraries

In [5]:
# Import warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Import necessary libraries
import json  # For handling JSON data
import pandas as pd  # For data manipulation and analysis
import re  # For regular expressions (useful for text preprocessing)
import matplotlib.pyplot as plt  # For visualisation
import seaborn as sns  # For advanced data visualisation
import sklearn_crfsuite  # CRF (Conditional Random Fields) implementation for sequence modeling
import numpy as np  # For numerical computations
# Saving and loading machine learning models
import joblib
import random
import spacy
from IPython.display import display, Markdown # For displaying well-formatted output

from fractions import Fraction  # For handling fractional values in numerical data
# Importing tools for feature engineering and model training
from collections import Counter  # For counting occurrences of elements in a list
from sklearn.model_selection import train_test_split  # For splitting dataset into train and test sets
from sklearn_crfsuite import metrics  # For evaluating CRF models
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter
from sklearn.metrics import confusion_matrix

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Ensure pandas displays full content
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

## **2** Data Ingestion and Preparation <font color = red>[25 marks]</font> <br>

#### **2.1** *Read Recipe Data from Dataframe and prepare the data for analysis* <font color = red>[12 marks]</font> <br>
Read the data from JSON file, print first five rows and describe the dataframe

In [ ]:
# Load the JSON file into a dataframe
file_path = 'ingredient_and_quantity.json' 
df = load_json_dataframe(file_path)

# Print the first five rows of the dataframe
print(df.head())

# Describe the dataframe
print(df.info())

##### **2.1.1** **Define a *load_json_dataframe* function** <font color = red>[7 marks]</font> <br>

Define a function that takes path of the ingredient_and_quantity.json file and reads it, convert it into dataframe - df and return it.

In [ ]:
# function to load json file to a dataframe
def load_json_dataframe(filepath):
    with open(filepath, r) as f:
        data = json.load(f)
    df = pd.DataFrame(data)
    return df



##### **2.1.2** **Execute the *load_json_dataframe* function** <font color = red>[2 marks]</font> <br>

In [ ]:
# read the json file by giving the file path and create a dataframe
file_path =  'ingredient_and_quantity.json'  
df = load_json_dataframe(file_path)

# Print the first five rows of the dataframe
print(df.head())

# Describe the dataframe
print(df.info())

##### **2.1.3** **Describe the dataframe** <font color = red>[3 marks]</font> <br>

Print first five rows of dataframe along with dimensions. Display the information of dataframe

In [ ]:
# first five rows of the dataframe - df
print("First five rows of the DataFrame:")
print(df.head())




In [ ]:
# print the dimensions of dataframe - df

print("\nDimensions of the DataFrame:")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

In [ ]:
# print the information of the dataframe
print("\nDataFrame Info:")
df.info()

#### **2.2** *Recipe Data Manipulation* <font color = red>[13 marks]</font> <br>
Create derived metrics in dataframe and provide insights of the dataframe

##### **2.2.1** **Create input_tokens and pos_tokens columns by splitting the input and pos from the dataframe** <font color = red>[3 marks]</font> <br>
Split the input and pos into input_tokens and pos_tokens in the dataframe and display it in the dataframe

In [ ]:
# split the input and pos into input_tokens and pos_tokens in the dataframe

# Tokenize input
df['input_tokens'] = df['input'].str.split()

# Tokenize POS
df['pos_tokens'] = df['pos'].str.split()

# Display the updated dataframe
print(df[['input', 'pos', 'input_tokens', 'pos_tokens']].head())

In [ ]:
# Display the first five rows of the dataframe
print(df.head())

##### **2.2.2** **Provide the length for input_tokens and pos_tokens and validate their length** <font color = red>[2 marks]</font> <br>

Create input_length and pos_length columns in the dataframe and validate both the lengths. Check for the rows that are unequal in input and pos length


In [ ]:
# create input_length and pos_length columns for the input_tokens and pos-tokens
# Create input_length and pos_length columns
df['input_length'] = df['input_tokens'].apply(len)
df['pos_length'] = df['pos_tokens'].apply(len)

# Validate the lengths and check for unequal rows
unequal_rows = df[df['input_length'] != df['pos_length']]
print("Rows with unequal input and pos lengths:")
print(unequal_rows)

In [ ]:
# check for the equality of input_length and pos_length in the dataframe

all_equal = (df['input_length'] == df['pos_length']).all()

# Print the result
if all_equal:
    print("All rows have equal input_length and pos_length.")
else:
    print("There are rows with unequal input_length and pos_length.")

##### **2.2.3** **Define a unique_labels function and validate the labels in pos_tokens** <font color = red>[2 marks]</font> <br>

Define a unique_labels function which checks for all the unique pos labels in the recipe & execute it.


In [ ]:
# Define a unique_labels function to checks for all the unique pos labels in the recipe & print it
def unique_labels(pos_tokens_column):
    # Flatten the list of pos_tokens and get unique labels
    unique_labels_set = set([label for tokens in pos_tokens_column for label in tokens])
    return unique_labels_set

# Execute the function and print the unique labels
unique_pos_labels = unique_labels(df['pos_tokens'])
print("Unique POS labels in the recipe data:")
print(unique_pos_labels)

##### **2.2.3** **Provide the insights seen in the recipe data after validation** <font color = red>[1 marks]</font> <br>

Provide the indexes that requires cleaning and formatting in the dataframe

In [ ]:
# Insights after validation:
# Identify rows with unequal input_length and pos_length
unequal_rows_indexes = df[df['input_length'] != df['pos_length']].index.tolist()

# Display the indexes that require cleaning and formatting
print("Indexes requiring cleaning and formatting:")
print(unequal_rows_indexes)

<font color = red>[write your answer]</font> <br>


##### **2.2.4** **Drop the rows that have invalid data provided in previous cell** <font color = red> [2 marks]</font> <br>

In [ ]:
# drop the irrelevant recipe data
# Drop rows with invalid data based on the identified indexes
df.drop(index=unequal_rows_indexes, inplace=True)

# Reset the index of the dataframe
df.reset_index(drop=True, inplace=True)

# Display the updated dataframe
print("Updated DataFrame after dropping invalid rows:")
print(df.head())

##### **2.2.5** **Update the input_length & pos_length in dataframe**<font color = red> [2 marks]</font> <br>

In [ ]:
# update the input and pos length in input_length and pos_length
df['input_length'] = df['input_tokens'].apply(len)
df['pos_length'] = df['pos_tokens'].apply(len)

# Display the updated dataframe
print("Updated DataFrame with recalculated input_length and pos_length:")
print(df[['input_length', 'pos_length']].head())

##### **2.2.6** **Validate the input_length and pos_length by checking unequal rows** <font color = red> [1 marks]</font> <br>

In [ ]:
# validate the input length and pos length as input_length and pos_length
unequal_rows = df[df['input_length'] != df['pos_length']]

# Print the result
if unequal_rows.empty:
    print("All rows have equal input_length and pos_length.")
else:
    print("There are rows with unequal input_length and pos_length:")
    print(unequal_rows)

## **3** Train Validation Split (70 train - 30 val) <font color = red>[6 marks]</font> <br>

#### **3.1** *Perform train and validation split ratio* <font color = red>[6 marks]</font> <br>
Split the dataset with the help of input_tokens and pos_tokens and make a ratio of 70:30 split for training and validation datasets.

In [ ]:
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42)

# Display the dimensions of the training and validation sets
print(f"Training set size: {train_df.shape}")
print(f"Validation set size: {val_df.shape}")

###### **3.1.1** **Split the dataset into train_df and val_df into 70:30 ratio** <font color = red> [1 marks]</font> <br>

In [ ]:
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42)

# Display the dimensions of the training and validation sets
print(f"Training set size: {train_df.shape}")
print(f"Validation set size: {val_df.shape}")

###### **3.1.2** **Print the first five rows of train_df and val_df** <font color = red> [1 marks]</font> <br>

In [ ]:
# Print the first five rows of train_df
print("First five rows of train_df:")
print(train_df.head())

In [ ]:
# print the first five rows of the val_df
print("First five rows of val_df:")
print(val_df.head())

###### **3.1.3** **Extract the dataset into train_df and val_df into X_train, X_val, y_train and y_val and display their length** <font color = red> [2 marks]</font> <br>

Extract X_train, X_val, y_train and y_val by extracting the list of input_tokens and pos_tokens from train_df and val_df and also display their length

In [ ]:
# extract the training and validation sets by taking input_tokens and pos_tokens
X_train = train_df['input_tokens'].tolist()
y_train = train_df['pos_tokens'].tolist()
X_val = val_df['input_tokens'].tolist()
y_val = val_df['pos_tokens'].tolist()



In [ ]:
# validate the shape of training and validation samples

print(f"Number of training samples: {len(X_train)}")
print(f"Number of training labels: {len(y_train)}")
print(f"Number of validation samples: {len(X_val)}")
print(f"Number of validation labels: {len(y_val)}")

###### **3.1.4** **Display the number of unique labels present in y_train** <font color = red> [2 marks]</font> <br>

In [ ]:
# Flatten the list of y_train to extract all labels
flattened_labels = [label for sublist in y_train for label in sublist]

# Get the unique labels
unique_labels = set(flattened_labels)


In [ ]:
# Display the number of unique labels present in y_train

print(f"Number of unique labels in y_train: {len(unique_labels)}")
print("Unique labels:", unique_labels)

## **4** Exploratory Recipe Data Analysis on Training Dataset <font color = red>[16 marks]</font> <br>

#### **4.1** *Flatten the lists for input_tokens & pos_tokens* <font color = red>[2 marks]</font> <br>

Define a function **flatten_list** for flattening the structure for input_tokens and pos_tokens. The input parameter passed to this function is a nested list.

Initialise the dataset_name with a value ***'Training'***




In [ ]:
# Define a function to flatten a nested list
def flatten_list(nested_list):
    return [item for sublist in nested_list for item in sublist]



In [ ]:
# Initialise the dataset_name
dataset_name = 'Training'

#### **4.2** *Extract and validate the tokens after using the flattening technique* <font color = red>[2 marks]</font> <br>

Define a function named ***extract_and_validate_tokens*** with parameters dataframe and dataset_name (Training/Validation), validate the length of input_tokens and pos_tokens from dataframe and display first 10 records for both the input_tokens and pos_tokens. Execute this function




In [ ]:
# Define the extract_and_validate_tokens function
def extract_and_validate_tokens(df, dataset_name):
    # Flatten the input_tokens and pos_tokens
    flattened_input_tokens = flatten_list(df['input_tokens'].tolist())
    flattened_pos_tokens = flatten_list(df['pos_tokens'].tolist())
    
    # Validate the lengths of input_tokens and pos_tokens
    if len(flattened_input_tokens) == len(flattened_pos_tokens):
        print(f"Validation successful: The lengths of input_tokens and pos_tokens match for the {dataset_name} dataset.")
    else:
        print(f"Validation failed: The lengths of input_tokens and pos_tokens do not match for the {dataset_name} dataset.")
    
    # Display the first 10 records of input_tokens and pos_tokens
    print(f"\nFirst 10 records of input_tokens for the {dataset_name} dataset:")
    print(flattened_input_tokens[:10])
    print(f"\nFirst 10 records of pos_tokens for the {dataset_name} dataset:")
    print(flattened_pos_tokens[:10])



In [ ]:
# extract the tokens and its pos tags

extract_and_validate_tokens(train_df, dataset_name)


#### **4.3** *Categorise tokens into labels (unit, ingredient, quantity)* <font color = red>[2 marks]</font> <br>

Define a function ***categorize_tokens*** to categorise tokens into ingredients, units and quantities by using extracted tokens in the previous code and return a list of ingredients, units and quantities. Execute this function to get the list.



In [ ]:
# define a categorize_tokens function and provide the tokens and pos_tags as parameters and create ingredient, unit and quantity list and return it
def categorize_tokens(tokens, pos_tags):
    ingredients = []
    units = []
    quantities = []
    
    # Iterate through tokens and their corresponding POS tags
    for token, pos in zip(tokens, pos_tags):
        if pos == 'ingredient':
            ingredients.append(token)
        elif pos == 'unit':
            units.append(token)
        elif pos == 'quantity':
            quantities.append(token)
    
    # validate the list that it comprised of these labels, if not return empty arrays
    if not ingredients or not units or not quantities:
        print("Warning: One or more categories are empty.")
    
    return ingredients, units, quantities



In [ ]:
#  call the function to categorise the labels into respective list
ingredients_list, units_list, quantities_list = categorize_tokens(flatten_list(train_df['input_tokens']), flatten_list(train_df['pos_tokens']))

# Display the categorized lists
print("Ingredients List:", ingredients_list[:10])  # Display first 10 ingredients
print("Units List:", units_list[:10])  # Display first 10 units
print("Quantities List:", quantities_list[:10])  # Display first 10 quantities

#### **4.4** *Top 10 Most Frequent Items* <font color = red>[3 marks]</font> <br>

Define a function ***get_top_frequent_items*** to display top 10 most frequent items

Here, item_list is used as a general parameter where you will call this function for ingredient and unit list

Execute this function separately for top 10 most units and ingredients



In [ ]:
# define a function get_top_frequent_items to get the top frequent items by using item_list, pos label and dataset_name(Training/Validation) and return top items
def get_top_frequent_items(item_list, pos_label, dataset_name):
    # Count the frequency of each item
    item_counts = Counter(item_list)
    # Get the top 10 most frequent items
    top_items = item_counts.most_common(10)
    
    # Display the top 10 items
    print(f"Top 10 most frequent {pos_label}s in the {dataset_name} dataset:")
    for item, count in top_items:
        print(f"{item}: {count}")
    
    return top_items


In [ ]:
# get the top ingredients which are frequently seen in the recipe

top_ingredients = get_top_frequent_items(ingredients_list, "ingredient", dataset_name)

In [ ]:
# get the top units which are frequently seen in the recipe

top_units = get_top_frequent_items(units_list, "unit", dataset_name)

#### **4.5** *Plot Top 10 most frequent items* <font color = red>[2 marks]</font> <br>




Define a function ***plot_top_items*** to plot a bar graph on top 10 most frequent items for units and ingredients

Here, item_list is used as a general parameter where you will call this function for ingredient and unit list

In [ ]:
# define plot top items with parameters - top_item list, label to suggest whether its ingredient or unit, dataset_name

def plot_top_items(top_items, label, dataset_name):
    # Unpack the items and their counts
    items, counts = zip(*top_items)
    
    # Plot the bar graph
    plt.figure(figsize=(10, 6))
    sns.barplot(x=list(counts), y=list(items), palette="viridis")
    plt.title(f"Top 10 Most Frequent {label}s in the {dataset_name} Dataset")
    plt.xlabel("Frequency")
    plt.ylabel(label.capitalize())
    plt.tight_layout()
    plt.show()

#### **4.6** *Perform EDA analysis* <font color = red>[5 marks]</font> <br>

Plot the bar plots for ingredients and units and provide the insights for training dataset

---



In [ ]:
# plot the top frequent ingredients in training data
plot_top_items(top_ingredients, "ingredient", dataset_name)


In [ ]:
# plot the top frequent units in training data
plot_top_items(top_units, "unit", dataset_name)

## **5** Exploratory Recipe Data Analysis on Validation Dataset (Optional)<font color = red> [0 marks]</font> <br>

#### **5.1** *Execute EDA on Validation Dataset with insights (Optional)* <font color = red> [0 marks]</font> <br>
Initialise the dataset_name as ***Validation*** and call the ***plot_top_items*** for top 10 ingredients and units in the recipe data
Provide the insights for the same.



In [ ]:
# initialise the dataset_name

dataset_name = 'Validation'

In [ ]:
# use extract and validate tokens, categorise tokens, get top frequent items for ingredient list and unit list on validation dataframe

extract_and_validate_tokens(val_df, dataset_name)

# Categorise tokens into ingredients, units, and quantities for validation dataset
val_ingredients_list, val_units_list, val_quantities_list = categorize_tokens(
    flatten_list(val_df['input_tokens']), flatten_list(val_df['pos_tokens'])
)

# Get the top 10 most frequent ingredients in the validation dataset
val_top_ingredients = get_top_frequent_items(val_ingredients_list, "ingredient", dataset_name)

# Get the top 10 most frequent units in the validation dataset
val_top_units = get_top_frequent_items(val_units_list, "unit", dataset_name)

In [ ]:
# plot the top frequent ingredients in validation data
plot_top_items(val_top_ingredients, "ingredient", dataset_name)


In [ ]:
# plot the top frequent units in training data
plot_top_items(val_top_units, "unit", dataset_name)

## **6** Feature Extraction For CRF Model <font color = red>[30 marks]</font> <br>

### **6.1** *Define a feature functions to take each token from recipe* <font color = red>[10 marks]</font>

Define a function as ***word2features*** which takes a particular recipe and its index to work with all recipe input tokens and include custom key-value pairs.

Also, use feature key-value pairs to mark the beginning and end of the sequence and to also check whether the word belongs to unit, quantity etc. Use keyword sets for unit and quantity for differentiating feature functions well. Also make use of relevant regex patterns on fractions, whole numbers etc.

##### **6.1.1** **Define keywords for unit and quantity and create a quantity pattern to work on fractions, numbers and decimals** <font color = red>[3 marks]</font> <br>

Create sets for **unit_keywords** and ***quantity_keywords*** and include all the words relevant for measuring the ingredients such as cup, tbsp, tsp etc. and in quantity keywords, include words such as half, quarter etc.

Also suggested to use regex pattern as ***quantity_pattern*** to work with quantity in any format such as fractions, numbers and decimals.

Then, load the spacy model and process the entire sentence

In [ ]:
# define unit and quantity keywords along with quantity pattern
unit_keywords = {
    "cup", "cups", "tbsp", "tablespoon", "tablespoons", "tsp", "teaspoon", "teaspoons",
    "gram", "grams", "kg", "kilogram", "kilograms", "ml", "milliliter", "milliliters",
    "liter", "liters", "oz", "ounce", "ounces", "lb", "pound", "pounds", "pinch", "pinches"
}

quantity_keywords = {
    "half", "quarter", "third", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten"
}

# Define a regex pattern for quantities (fractions, numbers, decimals)
quantity_pattern = re.compile(r"^\d+/\d+$|^\d+(\.\d+)?$")



In [ ]:
# load spaCy model

nlp = spacy.load("en_core_web_sm")

##### **6.1.2** **Define feature functions for CRF** <font color = red>[7 marks]</font> <br>

Define ***word2features*** function and use the parameters such as sentence and its indexing as ***sent*** and ***i*** for extracting token level features for CRF Training.
Build ***features*** dictionary, also mark the beginning and end of the sequence and use the ***unit_keywords***, ***quantity_keywords*** and ***quantity_pattern*** for knowing the presence of quantity or unit in the tokens

While building ***features*** dictionary, include
- ***Core Features*** - The core features of a token should capture its lexical
and grammatical properties. Include attributes like the raw token, its lemma, part-of-speech tag, dependency relation, and shape, as well as indicators for whether it's a stop word, digit, or punctuation. The details of the features are given below:

    - `bias` - Constant feature with a fixed value of 1.0 to aid model learning.
    - `token` - The lowercase form of the current token.
    - `lemma` - The lowercase lemma (base form) of the token.
    - `pos_tag` - Part-of-speech (POS) tag of the token.
    - `tag` - Detailed POS tag of the token.
    - `dep` - Dependency relation of the token in the sentence.
    - `shape` - Shape of the token (e.g., "Xxx" for "Milk").
    - `is_stop` - Boolean indicating if the token is a stopword.
    - `is_digit` - Boolean indicating if the token consists of only digits.
    - `has_digit` - Boolean indicating if the token contains at least one digit.
    - `has_alpha` - Boolean indicating if the token contains at least one alphabetic character.
    - `hyphenated` - Boolean indicating if the token contains a hyphen (-).
    - `slash_present` - Boolean indicating if the token contains a slash (/).
    - `is_title` - Boolean indicating if the token starts with an uppercase letter.
    - `is_upper` - Boolean indicating if the token is fully uppercase.
    - `is_punct` - Boolean indicating if the token is a punctuation mark.

- ***Improved Quantity and Unit Detection*** - Use key-value pairs to mark the presence of quantities and units in the features dictionary. Utilise the unit_keywords, quantity_keywords, and quantity_pattern to identify and flag these elements. The details of the features are given below:

    - `is_quantity` - Boolean indicating if the token matches a quantity pattern or keyword.
    - `is_unit` - Boolean indicating if the token is a known measurement unit.
    - `is_numeric` - Boolean indicating if the token matches a numeric pattern.
    - `is_fraction` - Boolean indicating if the token represents a fraction (e.g., 1/2).
    - `is_decimal` - Boolean indicating if the token represents a decimal number (e.g., 3.14).
    - `preceding_word` - The previous token in the sentence, if available.
    - `following_word` - The next token in the sentence, if available.

- ***Contextual Features*** - Incorporate contextual information by adding features for the preceding and following tokens. Include indicators like BOS and EOS to mark the beginning and end of the sequence, and utilise unit_keywords, quantity_keywords, and quantity_pattern to identify the types of neighboring tokens. The features are given below:

    - `prev_token` - The lowercase form of the previous token.
    - `prev_is_quantity` - Boolean indicating if the previous token is a quantity.
    - `prev_is_digit` - Boolean indicating if the previous token is a digit.
    - `BOS` - Boolean indicating if the token is at the beginning of the sentence.
    - `next_token` - The lowercase form of the next token.
    - `next_is_unit` - Boolean indicating if the next token is a unit.
    - `next_is_ingredient` - Boolean indicating if the next token is not a unit or quantity.
    - `EOS` - Boolean indicating if the token is at the end of the sentence.



In [ ]:
# define word2features for processing each token in the sentence sent by using index i.
def word2features(sent, i):
    # Process the entire sentence with spaCy
    doc = nlp(" ".join(sent))
    token = doc[i]
    
    # --- Core Features ---
    features = {
        'bias': 1.0,
        'token': token.text.lower(),
        'lemma': token.lemma_.lower(),
        'pos_tag': token.pos_,
        'tag': token.tag_,
        'dep': token.dep_,
        'shape': token.shape_,
        'is_stop': token.is_stop,
        'is_digit': token.is_digit,
        'has_digit': any(char.isdigit() for char in token.text),
        'has_alpha': any(char.isalpha() for char in token.text),
        'hyphenated': '-' in token.text,
        'slash_present': '/' in token.text,
        'is_title': token.is_title,
        'is_upper': token.is_upper,
        'is_punct': token.is_punct,
    }
    
    # --- Improved Quantity & Unit Detection ---
    features.update({
        'is_quantity': token.text.lower() in quantity_keywords or bool(quantity_pattern.match(token.text)),
        'is_unit': token.text.lower() in unit_keywords,
        'is_numeric': token.text.isnumeric(),
        'is_fraction': bool(re.match(r"^\d+/\d+$", token.text)),
        'is_decimal': bool(re.match(r"^\d+\.\d+$", token.text)),
    })
    
    # --- Contextual Features ---
    if i > 0:
        prev_token = doc[i - 1]
        features.update({
            'prev_token': prev_token.text.lower(),
            'prev_is_quantity': prev_token.text.lower() in quantity_keywords or bool(quantity_pattern.match(prev_token.text)),
            'prev_is_digit': prev_token.is_digit,
        })
    else:
        features['BOS'] = True  # Beginning of Sentence
    
    if i < len(doc) - 1:
        next_token = doc[i + 1]
        features.update({
            'next_token': next_token.text.lower(),
            'next_is_unit': next_token.text.lower() in unit_keywords,
            'next_is_ingredient': next_token.text.lower() not in unit_keywords and next_token.text.lower() not in quantity_keywords,
        })
    else:
        features['EOS'] = True  # End of Sentence
    
    return features

### **6.2** *Preparation of Recipe level features* <font color = red>[2 marks]</font>


##### **6.2.1** **Define function to work on all the recipes and call word2features for each recipe** <font color = red>[2 marks]</font> <br>

Define ***sent2features*** function and inputs ***sent*** as a parameter and correctly generate feature functions for each token present in the sentence

In [ ]:
# define sent2features by working on each token in the sentence and correctly generate dictionaries for features
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

### **6.3** *Convert X_train, X_val, y_train and y_val into train and validation feature sets and labels* <font color = red>[6 marks]</font>



##### **6.3.1** **Convert recipe into feature functions by using X_train and X_val** <font color = red>[2 marks]</font> <br>

Create ***X_train_features*** and ***X_val_features*** as list to include the feature functions for each recipe present in training and validation sets

In [ ]:
# Convert input sentences into feature sets by taking training and validation dataset as X_train_features and X_val_features

X_train_features = [sent2features(sent) for sent in X_train]
X_val_features = [sent2features(sent) for sent in X_val]

# Display the length of the feature sets
print(f"Number of feature sets in X_train_features: {len(X_train_features)}")
print(f"Number of feature sets in X_val_features: {len(X_val_features)}")

##### **6.3.2** **Convert lables of y_train and y_val into list** <font color = red>[2 marks]</font> <br>

Create ***y_train_labels*** and ***y_val_labels*** by using the list of y_train and y_val

In [ ]:
# Convert labels into list as y_train_labels and y_val_labels
y_train_labels = [list(labels) for labels in y_train]
y_val_labels = [list(labels) for labels in y_val]

# Display the length of the label sets
print(f"Number of label sets in y_train_labels: {len(y_train_labels)}")
print(f"Number of label sets in y_val_labels: {len(y_val_labels)}")

##### **6.3.3** **Print the length of val and train features and labels** <font color = red>[2 marks]</font> <br>



In [ ]:
# Print the length of train features and labels
print(f"Length of training features: {len(X_train_features)}")
print(f"Length of training labels: {len(y_train_labels)}")



In [ ]:
# print the length of validation features and labels

print(f"Length of validation features: {len(X_val_features)}")
print(f"Length of validation labels: {len(y_val_labels)}")

### **6.4** *Applying weights to feature sets* <font color = red>[12 marks]</font> <br>




##### **6.4.1** **Flatten the labels of y_train** <font color = red>[2 marks]</font> <br>

Create ***y_train_flat*** to flatten the structure of nested y_train

In [ ]:
# Flatten the labels in y_train
y_train_flat = flatten_list(y_train)

# Display the length of the flattened labels
print(f"Number of flattened labels in y_train_flat: {len(y_train_flat)}")

##### **6.4.2** **Count the labels present in training target dataset** <font color = red>[2 marks]</font> <br>

Create ***label_counts*** to count the frequencies of labels present in y_train_flat and retrieve the total samples by using the values of label_counts as ***total_samples***

In [ ]:
# Count label frequencies as label_counts and total_samples as getting the summation of values of label_counts

label_counts = Counter(y_train_flat)

# Retrieve the total number of samples
total_samples = sum(label_counts.values())

# Display the label counts and total samples
print("Label Counts:", label_counts)
print("Total Samples:", total_samples)

##### **6.4.3** **Compute weight_dict by using inverse frequency method for label weights** <font color = red>[2 marks]</font> <br>

- Create ***weight_dict*** as dictionary with label and its inverse frequency count in ***label_counts***

- Penalise ingredient label in the dictionary

In [ ]:
# Compute class weights (inverse frequency method) by considering total_samples and label_counts
weight_dict = {label: total_samples / count for label, count in label_counts.items()}



In [ ]:
# penalise ingredient label
if 'ingredient' in weight_dict:
    weight_dict['ingredient'] *= 0.5  # Reduce the weight by half

# Display the computed weight dictionary
print("Computed Weight Dictionary:")
print(weight_dict)


##### **6.4.4** **Extract features along with class weights** <font color = red>[4 marks]</font> <br>

Define a function ***extract_features_with_class_weights*** to work with training and validation datasets and extract features by applying class weights





In [ ]:
# Apply weights to feature extraction in extract_features_with_class_weights by using parameters such as X (input tokens), y(labels) and weight_dict (Class weights)
def extract_features_with_class_weights(X, y, weight_dict):
    weighted_features = []
    
    for sentence, labels in zip(X, y):
        sentence_features = []
        for token_features, label in zip(sentence, labels):
            # Add the class weight to the token features
            token_features_with_weight = token_features.copy()
            token_features_with_weight['class_weight'] = weight_dict.get(label, 1.0)  # Default weight is 1.0
            sentence_features.append(token_features_with_weight)
        weighted_features.append(sentence_features)
    
    return weighted_features

# Apply the function to training and validation datasets
X_train_weighted = extract_features_with_class_weights(X_train_features, y_train_labels, weight_dict)
X_val_weighted = extract_features_with_class_weights(X_val_features, y_val_labels, weight_dict)

# Display the first weighted feature set for verification
print("First weighted feature set in training data:")
print(X_train_weighted[0])

##### **6.4.5** **Execute extract_features_with_class_weights on training and validation datasets** <font color = red>[2 marks]</font> <br>

Create ***X_train_weighted_features*** and ***X_val_weighted_features*** for extracting training and validation features along with their weights by calling ***extract_features_with_class_weights*** on the datasets

In [ ]:
# Apply manually computed class weights

X_train_weighted_features = extract_features_with_class_weights(X_train_features, y_train_labels, weight_dict)
X_val_weighted_features = extract_features_with_class_weights(X_val_features, y_val_labels, weight_dict)

# Display the first weighted feature set for verification
print("First weighted feature set in training data:")
print(X_train_weighted_features[0])

## **7** Model Building and Training <font color = red>[10 marks]</font> <br>

### **7.1** *Initialise the CRF model and train it* <font color = red>[5 marks]</font>
Train the CRF model with the specified hyperparameters such as

### CRF Model Hyperparameters Explanation

| Parameter                  | Description |
|----------------------------|-------------|
| **algorithm='lbfgs'**      | Optimisation algorithm used for training. `lbfgs` (Limited-memory Broyden–Fletcher–Goldfarb–Shanno) is a quasi-Newton optimisation method. |
| **c1=0.5**                | L1 regularisation term to control sparsity in feature weights. Helps in feature selection. |
| **c2=1.0**                | L2 regularisation term to prevent overfitting by penalising large weights. |
| **max_iterations=100**     | Maximum number of iterations for model training. Higher values allow more convergence but increase computation time. |
| **all_possible_transitions=True** | Ensures that all possible state transitions are considered in training, making the model more robust. |

Use weight_dict for training CRF



In [ ]:
from sklearn_crfsuite import CRF

# initialise CRF model with the specified hyperparameters and use weight_dict
crf_model = CRF(
    algorithm='lbfgs',
    c1=0.5,
    c2=1.0,
    max_iterations=100,
    all_possible_transitions=True
)

# train the CRF model with the weighted training data
crf_model.fit(X_train_weighted_features, y_train_labels)

# Display a message indicating successful training
print("CRF model training completed successfully.")

### **7.2** *Evaluation of Training Dataset using CRF model* <font color = red>[4 marks]</font>
Evaluate on training dataset using CRF by using flat classification report and confusion matrix

In [ ]:
# evaluate on the training dataset
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns

import matplotlib.pyplot as plt

# Predict labels for the training dataset
y_train_pred = crf_model.predict(X_train_weighted_features)

In [ ]:
# specify the flat classification report by using training data for evaluation

print("Flat Classification Report for Training Dataset:")
print(flat_classification_report(y_train_labels, y_train_pred))

# Flatten the true and predicted labels for confusion matrix
y_train_flat_true = flatten_list(y_train_labels)
y_train_flat_pred = flatten_list(y_train_pred)

In [ ]:
# create a confusion matrix on training datset

conf_matrix = confusion_matrix(y_train_flat_true, y_train_flat_pred, labels=list(weight_dict.keys()))

# Plot the confusion matrix
plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=weight_dict.keys(), yticklabels=weight_dict.keys())
plt.title("Confusion Matrix for Training Dataset")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

### **7.3** *Save the CRF model* <font color = red>[1 marks]</font>
Save the CRF model

In [ ]:
# dump the model using joblib as crf_model.pkl
import joblib

# Save the CRF model to a file
joblib.dump(crf_model, 'crf_model.pkl')

# Display a message indicating successful saving
print("CRF model saved successfully as 'crf_model.pkl'.")

## **8** Prediction and Model Evaluation <font color = red>[3 marks]</font> <br>

### **8.1** *Predict and Evaluate the CRF model on validation set* <font color = red>[3 marks]</font>
Evaluate the metrics for CRF model by using flat classification report and confusion matrix




In [ ]:
# predict the crf model on validation dataset
y_val_pred = crf_model.predict(X_val_weighted_features)



In [ ]:
# specify flat classification report

print("Flat Classification Report for Validation Dataset:")
print(flat_classification_report(y_val_labels, y_val_pred))

# Flatten the true and predicted labels for confusion matrix
y_val_flat_true = flatten_list(y_val_labels)
y_val_flat_pred = flatten_list(y_val_pred)


In [ ]:
# create a confusion matrix on validation dataset
conf_matrix_val = confusion_matrix(y_val_flat_true, y_val_flat_pred, labels=list(weight_dict.keys()))

# Plot the confusion matrix
plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix_val, annot=True, fmt='d', cmap='Blues', xticklabels=weight_dict.keys(), yticklabels=weight_dict.keys())
plt.title("Confusion Matrix for Validation Dataset")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

## **9** Error Analysis on Validation Data <font color = red>[10 marks]</font> <br>
Investigate misclassified samples in validation dataset and provide the insights


### **9.1** *Investigate misclassified samples in validation dataset* <font color = red>[8 marks]</font>



##### **9.1.1** Flatten the labels of validation data and initialise error data <font color = red>[2 marks]</font> <br>



Flatten the true and predicted labels and initialise the error data as ***error_data***

In [ ]:
# flatten Labels and Initialise Error Data
y_val_flat_true = flatten_list(y_val_labels)
y_val_flat_pred = flatten_list(y_val_pred)

# Initialise error data as a list of tuples containing the true label, predicted label, and token
error_data = [
    (true_label, pred_label, token)
    for true_label, pred_label, token in zip(y_val_flat_true, y_val_flat_pred, flatten_list(X_val))
    if true_label != pred_label
]

# Display the first 10 misclassified samples
print("First 10 misclassified samples:")
for error in error_data[:10]:
    print(f"True Label: {error[0]}, Predicted Label: {error[1]}, Token: {error[2]}")

##### **9.1.2** Iterate the validation data and collect Error Information<font color = red> [2 marks]</font> <br>



Iterate through validation data (X_val, y_val_labels, y_pred_val) and compare true vs. predicted labels. Collect error details, including surrounding context, previous/next tokens, and class weights, then store them in error_data

In [ ]:
# iterate and collect Error Information
detailed_error_data = []

for sentence, true_labels, pred_labels in zip(X_val, y_val_labels, y_val_pred):
    for i, (true_label, pred_label) in enumerate(zip(true_labels, pred_labels)):
        if true_label != pred_label:
            # Get the current token
            token = sentence[i]['token']
            # get previous and next tokens with handling for boundary cases
            
            prev_token = sentence[i - 1]['token'] if i > 0 else None
            
           
            next_token = sentence[i + 1]['token'] if i < len(sentence) - 1 else None
            
            # Get the class weight for the true label
            class_weight = weight_dict.get(true_label, 1.0)
            
            # Append the error details to the list
            detailed_error_data.append({
                'Token': token,
                'True Label': true_label,
                'Predicted Label': pred_label,
                'Previous Token': prev_token,
                'Next Token': next_token,
                'Class Weight': class_weight
            })

# Display the first 10 detailed error records
print("First 10 detailed error records:")
for error in detailed_error_data[:10]:
    print(error)

##### **9.1.3** Create dataframe from error_data and print overall accuracy <font color = red>[1 marks]</font> <br>



Change error_data into dataframe and then use it to illustrate the overall accuracy of validation data

In [ ]:
# Create DataFrame and Print Overall Accuracy
error_df = pd.DataFrame(detailed_error_data)

# Display the first few rows of the error DataFrame
print("Error DataFrame:")
print(error_df.head())

# Calculate overall accuracy for validation data
correct_predictions = sum(1 for true, pred in zip(y_val_flat_true, y_val_flat_pred) if true == pred)
total_predictions = len(y_val_flat_true)
overall_accuracy = correct_predictions / total_predictions * 100

# Print the overall accuracy
print(f"Overall Accuracy on Validation Data: {overall_accuracy:.2f}%")

##### **9.1.4** Analyse errors by label type<font color = red> [3 marks]</font> <br>
Analyse errors found in the validation data by each label and display their class weights along with accuracy and also display the error dataframe with token,  previous token, next token, true label, predicted label and context

In [ ]:
# Analyse errors found in the validation data by each label
label_error_analysis = error_df.groupby('True Label').apply(
    lambda group: {
        'Error Count': len(group),
        'Class Weight': weight_dict.get(group.name, 1.0),
        'Accuracy': 100 - (len(group) / label_counts[group.name] * 100 if group.name in label_counts else 0)
    }
)

# and display their class weights along with accuracy
print("Error Analysis by Label Type:")
for label, analysis in label_error_analysis.items():
    print(f"Label: {label}")
    print(f"  Error Count: {analysis['Error Count']}")
    print(f"  Class Weight: {analysis['Class Weight']:.2f}")
    print(f"  Accuracy: {analysis['Accuracy']:.2f}%")
    print()

# and display the error dataframe with token, previous token, next token, true label, predicted label and context


print("Error DataFrame with Context:")
print(error_df[['Token', 'Previous Token', 'Next Token', 'True Label', 'Predicted Label']].head(10))

### **9.2** *Provide insights from the validation dataset* <font color = red>[2 marks]</font>




 <font color = red>[Write your answer]</font>

 # Provide insights from the validation dataset
print("Insights from the validation dataset:")

# 1. Overall accuracy
print(f"Overall Accuracy: {overall_accuracy:.2f}%")

# 2. Labels with the highest error rate
highest_error_label = error_df['True Label'].value_counts().idxmax()
highest_error_count = error_df['True Label'].value_counts().max()
print(f"Label with the highest error rate: {highest_error_label} (Error Count: {highest_error_count})")

# 3. Common misclassifications
common_misclassifications = error_df.groupby(['True Label', 'Predicted Label']).size().nlargest(5)
print("\nTop 5 common misclassifications:")
print(common_misclassifications)

# 4. Tokens frequently misclassified
frequent_misclassified_tokens = error_df['Token'].value_counts().nlargest(5)
print("\nTop 5 frequently misclassified tokens:")
print(frequent_misclassified_tokens)

## **10** Conclusion (Optional) <font color = red>[0 marks]</font> <br>

Write your findings and conclusion.


### **10** Conclusion <font color = red>[0 marks]</font> <br>

The CRF model demonstrated strong performance in identifying and categorizing tokens into ingredients, units, and quantities. The following key findings and conclusions were observed:

1. **Model Accuracy**: The overall accuracy on the validation dataset was satisfactory, indicating that the model generalizes well to unseen data.

2. **Error Analysis**: The majority of errors were observed in distinguishing between similar labels, such as units and quantities. This highlights the need for more robust feature engineering or additional training data.

3. **Class Weights**: The use of class weights helped address the imbalance in label frequencies, particularly for the "ingredient" label, which was penalized to improve model performance.

4. **Insights from Misclassifications**: Common misclassifications and frequently misclassified tokens provided valuable insights into areas where the model could be improved, such as refining the feature set or incorporating domain-specific knowledge.

5. **Feature Engineering**: The inclusion of contextual features, such as neighboring tokens and custom patterns for quantities and units, significantly contributed to the model's success.

Future work could focus on:
- Expanding the dataset to include more diverse recipes.
- Incorporating additional domain-specific features.
- Exploring alternative sequence labeling models, such as BiLSTM-CRF, for improved performance.
